## Package Import

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from ipywidgets import IntSlider, HBox, interactive_output
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import skew, kurtosis
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
import seaborn as sns
from functools import reduce
import talib
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from analysis.utils_analysis import *

***
### Notes
This notebook is aimed for the data restructure (pipeline) in order to be compatible with the future monthly return data classifiers. For more details on how we visualized and manipulated the broad assets data, please refer to the `initial_analysis.ipynb` notebook (i.e., section `Initial Visualizations`, `Aggregating Returns up to Weekly`). Now, we would like to use the weekly returns data file directly from the /data dir: `broad_assets_weekly_rets.xlsx`:
***

## Universe of Broad Asset Data Import

In [2]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_file = data_path + '/fpi_raw_data.xlsx'
data = pd.read_excel(data_file, sheet_name='Universe of broad assets', index_col=0, parse_dates=True)
broad_assets = data.copy()
broad_assets

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.0152,0.0757,-0.0023,0.0009,0.0422,0.0050,0.0373,0.0021,-0.0025,-0.0084,-0.0318
2000-08-01,0.0099,-0.0491,0.0079,0.0063,-0.0115,0.0404,-0.0292,0.0107,0.0096,0.0005,0.0146
2000-08-02,0.0007,-0.0178,0.0015,0.0037,0.0098,-0.0095,-0.0005,0.0010,0.0112,0.0012,0.0187
2000-08-03,0.0192,0.0761,0.0072,0.0031,-0.0031,-0.0210,-0.0307,-0.0095,0.0061,-0.0262,0.0072
2000-08-04,0.0142,-0.0030,0.0066,0.0062,0.0164,-0.0249,0.0129,0.0042,-0.0012,-0.0036,0.0326
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.0203,0.0258,-0.0144,-0.0047,0.0135,0.0194,0.0043,0.0039,0.0070,-0.0145,-0.0239
2024-09-03,-0.0423,-0.0631,0.0225,0.0078,-0.0618,0.0019,-0.0384,-0.0399,0.0028,-0.0086,-0.0392
2024-09-04,-0.0033,-0.0040,0.0250,0.0097,-0.0039,0.0043,-0.0051,-0.0007,-0.0092,0.0020,-0.0186


## Weekly Data Import

In [3]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_prefix = '/broad_assets_'
weekly_rets_file = data_path + data_prefix + 'weekly_rets.xlsx'
weekly_rets = pd.read_excel(weekly_rets_file, index_col=0)
weekly_rets

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.0152,0.0757,-0.0023,0.0009,0.0422,0.0050,0.0373,0.0021,-0.0025,-0.0084,-0.0318
2000-08-07,0.0679,0.0526,0.0159,0.0134,0.0365,-0.0237,-0.0196,-0.0116,0.0241,-0.0266,0.0346
2000-08-14,0.0157,0.0023,0.0152,0.0063,0.0171,0.0254,0.0294,0.0369,0.0085,0.0063,0.0481
2000-08-21,0.0095,0.0566,-0.0018,-0.0003,0.0068,-0.0264,0.0016,-0.0024,0.0077,0.0018,0.0384
2000-08-28,0.0184,0.0653,0.0019,0.0041,0.0379,-0.0353,0.0195,-0.0170,0.0019,-0.0095,0.0313
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.0607,0.0718,-0.0440,-0.0147,0.0210,0.0425,0.0431,0.0882,0.0093,0.0503,0.0583
2024-08-19,0.1006,0.1346,0.0199,-0.0004,0.1037,0.0321,0.0931,0.0771,-0.0238,0.0239,-0.0377
2024-08-26,0.0020,-0.0266,0.0028,0.0064,0.0450,0.0540,0.0290,-0.0180,-0.0195,0.0097,0.0332


*** 
## Equity Domestic / Global Data Import

In [4]:
data_path = os.path.dirname(os.getcwd()) + '/data'
equity_domestic = pd.read_excel(data_path + '/equity_domestic_monthly_rets.xlsx', index_col=0, parse_dates=True)
equity_global = pd.read_excel(data_path + '/equity_global_monthly_rets.xlsx', index_col=0, parse_dates=True)
equity_domestic_monthly_rets = pd.read_excel(data_path + '/equity_domestic_monthly_rets.xlsx', index_col=0, parse_dates=True)
equity_global_monthly_rets = pd.read_excel(data_path + '/equity_global_monthly_rets.xlsx', index_col=0, parse_dates=True)
equity_domestic

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 37,Asset 38,Asset 39,Asset 40,Asset 41,Asset 42,Asset 43,Asset 44,Asset 45,Asset 46
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-03,0.0000,0.0000,0.0000,-0.0338,0.0000,0.0000,0.0000,-0.0137,0.0000,0.0000,...,0.0115,0.0419,0.0675,0.1113,0.0419,0.0177,-0.0411,-0.0118,-0.0171,-0.0013
2001-02-01,0.0000,0.0000,0.0000,-0.0378,0.0000,0.0000,0.0000,0.0048,0.0000,0.0000,...,0.0166,0.0215,-0.0046,0.0319,0.0253,-0.0520,-0.0496,0.0120,0.0114,0.0100
2001-03-01,0.0000,0.0000,0.0000,0.0269,0.0000,0.0000,0.0000,0.0441,0.0000,0.0000,...,-0.0172,0.0704,0.0395,-0.1746,0.0087,0.0011,0.0999,0.0110,0.0135,0.0058
2001-04-02,0.0000,0.0000,0.0000,-0.0210,0.0000,0.0000,0.0000,0.0367,0.0000,0.0000,...,-0.0038,-0.0238,-0.0028,-0.1542,-0.0019,-0.0201,0.0181,0.0008,-0.0176,0.0081
2001-05-01,0.0000,0.0000,0.0000,0.0414,0.0000,0.0000,0.0000,0.0314,0.0000,0.0000,...,0.0053,0.1430,0.0607,0.2815,0.0649,0.0244,0.0337,-0.0122,-0.0239,0.0032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,-0.0672,-0.0762,-0.0498,-0.0160,-0.0570,-0.0474,0.0096,-0.0149,-0.0978,-0.0203,...,-0.0728,-0.0724,-0.1059,-0.0831,-0.0480,-0.0287,0.0441,-0.0186,-0.0398,-0.0003
2024-06-03,0.0726,0.0105,0.0971,0.0548,0.0147,0.0757,0.0090,0.0565,-0.0070,-0.0625,...,0.0137,0.0037,0.0567,0.1348,0.0015,-0.0282,0.0741,0.0206,0.0378,0.0063
2024-07-01,-0.0249,-0.0829,-0.0115,0.0312,0.0068,0.0321,-0.0595,-0.0115,-0.1055,-0.0501,...,0.0166,0.0102,-0.0685,-0.0050,-0.0899,0.0122,-0.0617,-0.0019,-0.0153,0.0034


***
The input format of output monthly data below is banned. We still want our original whole dataset right now.

In [5]:
def natural_sort_key(asset_name):
    """
    Helper function to generate sorting key for natural sorting.
    Extracts numerical part of asset name for correct order (e.g., 'Asset 1', 'Asset 10').
    """
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', asset_name)]

def split_date(df, base_name="equity_dom_period"):
    """
    Splits data into multiple datasets based on the availability of asset data.
    Assumes 'Date' is already set as the index, ensuring columns are ordered as Asset 1, Asset 2, etc.
    Uses manual natural sorting to ensure assets like Asset 10 come after Asset 9, not Asset 1.
    Each split period is renamed according to a specified base name.

    Args:
    df (pd.DataFrame): The input dataframe with 'Date' as index and asset columns.
    base_name (str): Base name for each period key.

    Returns:
    dict: A dictionary where keys are renamed period identifiers and values are dataframes for each period.
    """
    
    periods = {}
    current_assets = set()
    period_start = df.index[0]
    
    for i in range(len(df)):
        row_assets = df.iloc[i].dropna().index
        
        # If the available assets change, finalize the current period and start a new one
        if set(row_assets) != current_assets:
            if current_assets:  # Only add a period if it's not the first iteration
                period_data = df[(df.index >= period_start) & (df.index < df.index[i])]
                sorted_columns = sorted(current_assets, key=natural_sort_key)
                period_data = period_data[sorted_columns]
                period_key = f"{base_name}_{len(periods) + 1}"
                periods[period_key] = period_data
            period_start = df.index[i]
            current_assets.update(row_assets)
    
    sorted_columns = sorted(current_assets, key=natural_sort_key)
    period_data = df[df.index >= period_start][sorted_columns]
    period_key = f"{base_name}_{len(periods) + 1}"
    periods[period_key] = period_data
    
    return periods

def export_split_dataframes(dataframes, prefix="equity_dom", output_folder="data"):
    """
    Exports each dataframe in the given dictionary to separate Excel files, with a customizable prefix.
    
    Args:
    dataframes (dict): Dictionary of dataframes to export.
    prefix (str): Prefix to use in naming each file (e.g., 'equity_glb' or 'equity_dom').
    output_folder (str): The folder where files will be saved.
    """
    data_path = os.path.dirname(os.getcwd()) + '/data'
    
    for i, (period_name, period_df) in enumerate(dataframes.items(), start=1):
        file_name = f"{prefix}_period_{i}.xlsx"
        file_path = os.path.join(data_path, file_name)
        period_df.to_excel(file_path)

In [6]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_file = data_path + '/fpi_raw_data.xlsx'
equity_domestic = pd.read_excel(data_file, sheet_name='Equity factor baskets domestic', index_col=0, parse_dates=True)
equity_global = pd.read_excel(data_file, sheet_name='Equity factor baskets global', index_col=0, parse_dates=True)
split_date_domestic = split_date(equity_domestic, base_name = 'equity_dom_period')
split_date_global = split_date(equity_global, base_name = 'equity_glb_period')
export_split_dataframes(split_date_domestic, prefix="equity_dom")
export_split_dataframes(split_date_global, prefix="equity_glb")

## Monthly Data Import

In [7]:
monthly_dom_returns = apply_monthly_returns(split_date_domestic)
monthly_glb_returns = apply_monthly_returns(split_date_global)

monthly_dom_returns_period_1 = monthly_dom_returns["equity_dom_period_1"]
monthly_dom_returns_period_2 = monthly_dom_returns["equity_dom_period_2"]
monthly_dom_returns_period_3 = monthly_dom_returns["equity_dom_period_3"]
monthly_dom_returns_period_4 = monthly_dom_returns["equity_dom_period_4"]

monthly_glb_returns_period_1 = monthly_glb_returns["equity_glb_period_1"]
monthly_glb_returns_period_2 = monthly_glb_returns["equity_glb_period_2"]
monthly_glb_returns_period_3 = monthly_glb_returns["equity_glb_period_3"]
monthly_glb_returns_period_4 = monthly_glb_returns["equity_glb_period_4"]
monthly_glb_returns_period_5 = monthly_glb_returns["equity_glb_period_5"]
monthly_glb_returns_period_6 = monthly_glb_returns["equity_glb_period_6"]
monthly_glb_returns_period_7 = monthly_glb_returns["equity_glb_period_7"]
monthly_glb_returns_period_8 = monthly_glb_returns["equity_glb_period_8"]
monthly_glb_returns_period_9 = monthly_glb_returns["equity_glb_period_9"]
monthly_glb_returns_period_10 = monthly_glb_returns["equity_glb_period_10"]
monthly_glb_returns_period_11 = monthly_glb_returns["equity_glb_period_11"]
monthly_dom_returns_period_4

NameError: name 'apply_monthly_returns' is not defined

In [8]:
data_path = os.path.dirname(os.getcwd()) + '/data'
equity_dom_monthly_rets_file = data_path + '/equity_domestic_monthly_rets.xlsx'
equity_glb_monthly_rets_file = data_path + '/equity_global_monthly_rets.xlsx'
equity_domestic_monthly_rets = pd.read_excel(equity_dom_monthly_rets_file, index_col=0)
equity_global_monthly_rets = pd.read_excel(equity_glb_monthly_rets_file, index_col=0)
display(equity_domestic_monthly_rets.head(), equity_global_monthly_rets.head())

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 37,Asset 38,Asset 39,Asset 40,Asset 41,Asset 42,Asset 43,Asset 44,Asset 45,Asset 46
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-03,0.0000,0.0000,0.0000,-0.0338,0.0000,0.0000,0.0000,-0.0137,0.0000,0.0000,...,0.0115,0.0419,0.0675,0.1113,0.0419,0.0177,-0.0411,-0.0118,-0.0171,-0.0013
2001-02-01,0.0000,0.0000,0.0000,-0.0378,0.0000,0.0000,0.0000,0.0048,0.0000,0.0000,...,0.0166,0.0215,-0.0046,0.0319,0.0253,-0.0520,-0.0496,0.0120,0.0114,0.0100
2001-03-01,0.0000,0.0000,0.0000,0.0269,0.0000,0.0000,0.0000,0.0441,0.0000,0.0000,...,-0.0172,0.0704,0.0395,-0.1746,0.0087,0.0011,0.0999,0.0110,0.0135,0.0058
2001-04-02,0.0000,0.0000,0.0000,-0.0210,0.0000,0.0000,0.0000,0.0367,0.0000,0.0000,...,-0.0038,-0.0238,-0.0028,-0.1542,-0.0019,-0.0201,0.0181,0.0008,-0.0176,0.0081
2001-05-01,0.0000,0.0000,0.0000,0.0414,0.0000,0.0000,0.0000,0.0314,0.0000,0.0000,...,0.0053,0.1430,0.0607,0.2815,0.0649,0.0244,0.0337,-0.0122,-0.0239,0.0032


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 43,Asset 44,Asset 45,Asset 46,Asset 47,Asset 48,Asset 49,Asset 50,Asset 51,Asset 52
Date,,,,,,,,,,,,,,,,,,,,,
2003-04-14,0.0059,0.0035,0.0000,0.0069,0.0000,0.0000,0.0090,0.0042,0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0028,-0.0009,-0.0028,0.0000,0.0000,0.0207,0.0204
2003-05-01,0.1023,0.1299,0.0000,0.0811,0.0000,0.0000,0.0997,0.0606,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0328,0.0056,0.0169,0.0000,0.0000,0.0332,0.0638
2003-06-02,0.0422,0.3734,0.0000,0.1223,0.0000,0.0000,0.1478,0.2189,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0603,0.0047,0.0370,0.0000,0.0000,0.0593,0.0683
2003-07-01,0.1001,0.1734,0.0000,0.0544,0.0000,0.0000,0.0618,0.1381,0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0115,0.0035,-0.0025,0.0000,0.0000,0.0158,0.0260
2003-08-01,0.0754,0.1196,0.0000,-0.0382,0.0000,0.0000,0.0560,0.0665,0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0922,-0.0049,-0.0515,0.0000,0.0000,-0.0002,0.0366


***
## Indicators Rebuild (270 for each asset)

#### ROC

In [9]:
def calculate_roc(data, periods):
    """
    Calculate ROC for each specified period and concatenate them into a single DataFrame.
    
    :param data: DataFrame containing return data for one asset
    :param periods: List of periods for calculating ROC
    :return: DataFrame with ROC indicators for each specified period
    """
    roc_dfs = []
    for period in periods:
        roc_df = data.rolling(window=period).apply(lambda x: np.prod(1 + x) - 1, raw=False)
        roc_df.columns = [f'ROC{period}_{col}' for col in data.columns]
        roc_dfs.append(roc_df)

    return pd.concat(roc_dfs, axis=1)

### MACD

In [10]:
def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    """
    Calculate MACD for each asset based on returns data.
    
    :param data: DataFrame containing return data for one asset
    :param short_period: Short-term EMA period (default 12)
    :param long_period: Long-term EMA period (default 26)
    :param signal_period: Signal line EMA period (default 9)
    :return: DataFrame with MACD line, Signal line, and MACD histogram
    """
    macd_df = pd.DataFrame(index=data.index)
    
    for col in data.columns:
        price_data = (1 + data[col]).cumprod()
        short_ema = price_data.ewm(span=short_period, adjust=False).mean()
        long_ema = price_data.ewm(span=long_period, adjust=False).mean()
        macd_line = short_ema - long_ema
        signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
        macd_histogram = macd_line - signal_line

        macd_df[f'MACD_Line_{col}'] = macd_line
        macd_df[f'Signal_Line_{col}'] = signal_line
        macd_df[f'MACD_Histogram_{col}'] = macd_histogram
    
    return macd_df


### RSI

In [11]:
def calculate_rsi(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a pandas DataFrame or Series that contains
    daily return data by first converting it to a cumulative price series.
    
    Parameters:
    data (pd.Series or pd.DataFrame): Daily returns data.
    window (int): The period for calculating RSI, default is 14.
    
    Returns:
    pd.DataFrame: RSI and Stochastic RSI values.
    """
    price_data = (1 + data).cumprod()
    delta = price_data.diff()
    
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    
    if isinstance(data, pd.DataFrame):
        gain = pd.DataFrame(gain, index=price_data.index, columns=price_data.columns)
        loss = pd.DataFrame(loss, index=price_data.index, columns=price_data.columns)
    else:
        gain = pd.Series(gain, index=price_data.index)
        loss = pd.Series(loss, index=price_data.index)

    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    if isinstance(rsi, pd.DataFrame):
        rsi.columns = [f'RSI_{col}' for col in rsi.columns]

    stoch_rsi = (rsi - rsi.rolling(window=window).min()) / (rsi.rolling(window=window).max() - rsi.rolling(window=window).min())

    if isinstance(stoch_rsi, pd.DataFrame):
        stoch_rsi.columns = [f'Stoch_{col}' for col in stoch_rsi.columns]
    
    rsi_df = pd.concat([rsi, stoch_rsi], axis=1)

    return rsi_df.dropna()


### Bollinger Bands

In [12]:
def calculate_bollinger_bands(data, windows=[20, 90, 180, 240], num_std=2):
    """
    Compute Bollinger Bands for multiple rolling window sizes based on return data
    by first converting returns to a cumulative price series.
    
    Parameters:
    - data (pd.DataFrame): A DataFrame of daily percent return data.
    - windows (list of int): List of rolling window sizes for moving averages.
    - num_std (int): Number of standard deviations for the bands (default is 2).

    Returns:
    - pd.DataFrame: Data with Bollinger Bands (MA, Upper, Lower) for each specified window.
    """
    price_data = (1 + data).cumprod()
    
    bollinger_dfs = []
    
    for window in windows:
        rolling_mean = price_data.rolling(window=window).mean()
        rolling_std = price_data.rolling(window=window).std()
        upper_band = rolling_mean + (rolling_std * num_std)
        lower_band = rolling_mean - (rolling_std * num_std)
        
        rolling_mean.columns = [f'SMA{window}_{col}' for col in data.columns]
        upper_band.columns = [f'Upper{window}_{col}' for col in data.columns]
        lower_band.columns = [f'Lower{window}_{col}' for col in data.columns]
        
        bollinger_df = pd.concat([rolling_mean, upper_band, lower_band], axis=1)
        bollinger_dfs.append(bollinger_df)
    
    return pd.concat(bollinger_dfs, axis=1).dropna()


### Hilbert Transform

In [13]:
def calc_hilbert(returns):
    data = (1 + returns).cumprod()
    ht_transform = pd.DataFrame()

    for asset in returns.columns:
        ht_transform['ht_dcperiod_' + asset] = talib.HT_DCPERIOD(data[asset])                   # Calculate Hilbert Transform - Dominant Cycle Period
        ht_transform['ht_dcphase_' + asset] = talib.HT_DCPHASE(data[asset])                     # Calculate Hilbert Transform - Dominant Cycle Phase
        ht_transform['inphase_' + asset], data['quadrature'] = talib.HT_PHASOR(data[asset])     # Calculate Hilbert Transform - Phasor Components
        ht_transform['sine_' + asset], data['leadsine'] = talib.HT_SINE(data[asset])            # Calculate Hilbert Transform - SineWave
        ht_transform['ht_trendmode_' + asset] = talib.HT_TRENDMODE(data[asset])                 # Calculate Hilbert Transform - Trend vs Cycle Mode

    return ht_transform.dropna()

### EWMA Volatility

In [14]:
def calc_ewma_volatility(excess_returns, asset_col, theta=0.94, initial_vol=0.2 / np.sqrt(252)):
    """
    Calculate EWMA volatility for a single asset.

    :param excess_returns: DataFrame containing returns for one asset
    :param asset_col: Name of the asset column
    :param theta: Smoothing parameter for EWMA (default 0.94)
    :param initial_vol: Initial volatility value (default 0.2 / sqrt(252))
    :return: DataFrame with EWMA volatility for the specified asset
    """
    var_t0 = initial_vol ** 2
    ewma_var = [var_t0]
    
    for d in excess_returns.index:
        new_ewma_var = ewma_var[-1] * theta + (excess_returns.loc[d, asset_col] ** 2) * (1 - theta)
        ewma_var.append(new_ewma_var)
    ewma_var.pop(0)  # Remove the initial var_t0
    ewma_vol = [np.sqrt(v) for v in ewma_var]
    
    return pd.DataFrame({f'EWMA_vol_{asset_col}': ewma_vol}, index=excess_returns.index)

### Rolling Volatility

In [15]:
def calculate_rolling_volatilities(data, windows=[20, 60, 126, 252, 504]):
    """
    Calculate rolling volatilities for multiple windows.

    :param data: DataFrame with daily returns for each asset
    :param windows: List of rolling window periods
    :return: DataFrame with rolling volatilities for each asset and window
    """
    rolling_vol_dfs = []
    for window in windows:
        rolling_vol_df = data.rolling(window=window).std()
        rolling_vol_df.columns = [f'RollingVol_{window}_{col}' for col in data.columns]
        rolling_vol_dfs.append(rolling_vol_df)
        
    return pd.concat(rolling_vol_dfs, axis=1).dropna()

### Rolling Skew

In [16]:
def calculate_rolling_skew(data, windows=[20, 60, 126, 252, 504]):
    """
    Calculate rolling skew for multiple windows.

    :param data: DataFrame with daily returns for each asset
    :param windows: List of rolling window periods
    :return: DataFrame with rolling skew for each asset and window
    """
    rolling_skew_dfs = []
    for window in windows:
        rolling_skew_df = broad_assets.rolling(window=window).skew()
        rolling_skew_df.columns = [f'RollingSkew_{window}_{col}' for col in data.columns]
        rolling_skew_dfs.append(rolling_skew_df)
        
    return pd.concat(rolling_skew_dfs, axis=1).dropna()

### Rolling Kurtosis

In [17]:
def calculate_rolling_kurt(data, windows=[20, 60, 126, 252, 504]):
    """
    Calculate rolling kurtosis for multiple windows.

    :param data: DataFrame with daily returns for each asset
    :param windows: List of rolling window periods
    :return: DataFrame with rolling kurtosis for each asset and window
    """
    rolling_kurt_dfs = []
    for window in windows:
        rolling_kurt_df = broad_assets.rolling(window=window).kurt()
        rolling_kurt_df.columns = [f'RollingKurt_{window}_{col}' for col in data.columns]
        rolling_kurt_dfs.append(rolling_kurt_df)
        
    return pd.concat(rolling_kurt_dfs, axis=1).dropna()

In [18]:
def apply_indicators(data, 
                     indicators, 
                     assets, 
                     reference_data, 
                     theta=0.94, 
                     initial_vol=0.2 / np.sqrt(252),
                     rolling_windows=[20, 60, 126, 252, 504],
                     bollinger_windows=[20, 90, 180, 240],
                     lags=[1, 2, 4, 6, 12]):
    """
    Apply indicators, rolling volatilities, expanding skew, and Hilbert Transform indicators for each asset.
    
    :param data: DataFrame with weekly or monthly returns for each asset
    :param indicators: Dictionary of indicator functions
    :param assets: List of asset columns to process
    :param reference_data: DataFrame of daily returns for calculations
    :param theta: Smoothing parameter for EWMA
    :param initial_vol: Initial volatility for EWMA calculation
    :param rolling_windows: List of windows for calculating rolling volatilities
    :param lags: List of lag periods to create lagged variables
    """

    # Calculate indicators (MACD, RSI, Bollinger Bands) using daily data in reference_data
    macd_df = calculate_macd(reference_data, short_period=12, long_period=26, signal_period=9)
    rsi_df = calculate_rsi(reference_data, window=14)
    bollinger_df = calculate_bollinger_bands(reference_data, windows=bollinger_windows)

    # Calculate rolling statistics using daily data
    rolling_vol_dfs = {window: reference_data.rolling(window=window).std() for window in rolling_windows}
    rolling_skew_dfs = {window: reference_data.rolling(window=window).skew() for window in rolling_windows}
    rolling_kurt_dfs = {window: reference_data.rolling(window=window).kurt() for window in rolling_windows}

    # Expanding skew/kurtosis using daily data
    expanding_skew_df = reference_data.expanding(min_periods=(252 * 2)).skew().dropna()
    expanding_kurt_df = reference_data.expanding(min_periods=(252 * 2)).kurt().dropna()

    # Hilbert Transform using daily data
    hilbert_df = calc_hilbert(reference_data)

    # Loop through each asset and calculate indicators
    asset_dataframes = {}
    for asset in assets:
        indicator_dfs = []

        # Get MACD, RSI, and Bollinger Bands indicators for the asset
        macd_asset_df = macd_df[[f'MACD_Line_{asset}', f'Signal_Line_{asset}', f'MACD_Histogram_{asset}']]
        rsi_asset_df = rsi_df[[f'RSI_{asset}', f'Stoch_RSI_{asset}']]
        bollinger_asset_df = bollinger_df[[f'SMA{window}_{asset}' for window in bollinger_windows] + 
                                          [f'Upper{window}_{asset}' for window in bollinger_windows] +
                                          [f'Lower{window}_{asset}' for window in bollinger_windows]]

        # Append indicators
        indicator_dfs.extend([macd_asset_df, rsi_asset_df, bollinger_asset_df])

        # Apply custom indicators from the indicators dictionary
        for indicator_name, indicator_func in indicators.items():
            indicator_dfs.append(indicator_func(data[[asset]]))

        # Calculate EWMA volatility for each asset
        ewma_vol_df = calc_ewma_volatility(reference_data, asset, theta=theta, initial_vol=initial_vol)
        indicator_dfs.append(ewma_vol_df)

        # Add rolling and expanding stats
        for window, df in rolling_vol_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingVol_{window}_{asset}'}))
        for window, df in rolling_skew_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingSkew_{window}_{asset}'}))
        for window, df in rolling_kurt_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingKurt_{window}_{asset}'}))
        indicator_dfs.extend([expanding_skew_df[[asset]].rename(columns={asset: f'ExpandingSkew_{asset}'}),
                              expanding_kurt_df[[asset]].rename(columns={asset: f'ExpandingKurt_{asset}'})])

        # Add Hilbert Transform for each asset
        indicator_dfs.append(hilbert_df[[f'ht_dcperiod_{asset}', f'ht_dcphase_{asset}', f'inphase_{asset}', 
                                         f'sine_{asset}', f'ht_trendmode_{asset}']])

        # Combine indicators and apply lags
        combined_df = pd.concat(indicator_dfs, axis=1).dropna()
        lagged_features = [combined_df.shift(lag).add_suffix(f'_lag{lag}') for lag in lags]
        combined_df = pd.concat([combined_df] + lagged_features, axis=1).dropna()

        # Add one-hot encoding for the asset and binary target variable (optional)
        one_hot_df = pd.DataFrame(0, index=combined_df.index, columns=[f'Asset_{i+1}' for i in range(len(assets))])
        one_hot_df[f'Asset_{assets.index(asset) + 1}'] = 1
        combined_df = pd.concat([combined_df, one_hot_df], axis=1)

        combined_df.columns = [re.sub(f'_Asset \d+', '', col) if not col.startswith('Asset_') else col
                               for col in combined_df.columns]      

        asset_dataframes[asset] = combined_df
    
    return asset_dataframes

***
## Indicators for Broad Assets

In [19]:
roc_periods = [4, 6, 12, 26, 52]
asset_columns = [col for col in weekly_rets.columns if col.startswith('Asset')]

indicators = {
    'ROC': lambda df: calculate_roc(df, roc_periods)
}

asset_dataframes = apply_indicators(
    data=weekly_rets,             # Weekly returns data for each asset
    indicators=indicators,        # Dictionary of indicators
    assets=asset_columns,         # List of asset columns
    reference_data=broad_assets   # Daily return data for indicators calculation
)

asset_dataframes[asset_columns[0]]

,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_2,Asset_3,Asset_4,Asset_5,Asset_6,Asset_7,Asset_8,Asset_9,Asset_10,Asset_11
Date,,,,,,,,,,,,,,,,,,,,,
2002-10-28,0.0067,0.0009,0.0058,68.4866,0.8951,0.3127,0.3499,0.4468,0.4815,0.3714,...,0,0,0,0,0,0,0,0,0,0
2002-11-04,0.0083,0.0052,0.0031,58.3816,0.3709,0.3264,0.3449,0.4410,0.4763,0.3802,...,0,0,0,0,0,0,0,0,0,0
2002-11-11,0.0073,0.0078,-0.0005,45.2142,0.0000,0.3427,0.3410,0.4357,0.4712,0.3647,...,0,0,0,0,0,0,0,0,0,0
2002-11-18,0.0059,0.0064,-0.0005,56.2893,0.4459,0.3455,0.3383,0.4293,0.4661,0.3648,...,0,0,0,0,0,0,0,0,0,0
2002-11-25,0.0097,0.0076,0.0022,55.9635,0.4966,0.3513,0.3390,0.4225,0.4613,0.3786,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,-0.1897,-0.1354,-0.0543,37.4974,0.6799,8.9749,8.6904,8.0864,7.5636,9.8747,...,0,0,0,0,0,0,0,0,0,0
2024-08-19,0.0200,-0.0798,0.0998,58.9321,1.0000,8.9141,8.7429,8.1629,7.6225,9.6944,...,0,0,0,0,0,0,0,0,0,0
2024-08-26,0.1371,0.0474,0.0897,80.0522,0.9280,9.0253,8.8285,8.2464,7.6900,9.9739,...,0,0,0,0,0,0,0,0,0,0


## Data Export for 11 Broad Assets

In [20]:
data_path = os.path.dirname(os.getcwd()) + '/data'
os.makedirs(data_path, exist_ok=True)

# Export each asset DataFrame to an Excel file
for i, (asset, df) in enumerate(asset_dataframes.items(), start=1):
    filename = os.path.join(data_path, f'broad_asset_{i}_features.xlsx')
    df.to_excel(filename, sheet_name=f'{asset}_features', index=True)
    print(f"Exported {asset} to {filename}")

Exported Asset 1 to /home/sam/projects/UniversityofChicago/data/broad_asset_1_features.xlsx
Exported Asset 2 to /home/sam/projects/UniversityofChicago/data/broad_asset_2_features.xlsx
Exported Asset 3 to /home/sam/projects/UniversityofChicago/data/broad_asset_3_features.xlsx
Exported Asset 4 to /home/sam/projects/UniversityofChicago/data/broad_asset_4_features.xlsx
Exported Asset 5 to /home/sam/projects/UniversityofChicago/data/broad_asset_5_features.xlsx
Exported Asset 6 to /home/sam/projects/UniversityofChicago/data/broad_asset_6_features.xlsx
Exported Asset 7 to /home/sam/projects/UniversityofChicago/data/broad_asset_7_features.xlsx
Exported Asset 8 to /home/sam/projects/UniversityofChicago/data/broad_asset_8_features.xlsx
Exported Asset 9 to /home/sam/projects/UniversityofChicago/data/broad_asset_9_features.xlsx
Exported Asset 10 to /home/sam/projects/UniversityofChicago/data/broad_asset_10_features.xlsx
Exported Asset 11 to /home/sam/projects/UniversityofChicago/data/broad_asset_1

***
## Indicators for Equity Domestic

In [21]:
def apply_indicators(data, 
                     indicators, 
                     assets, 
                     reference_data, 
                     theta=0.94, 
                     initial_vol=0.2 / np.sqrt(252),
                     rolling_windows=[20, 60, 126, 252, 504],
                     bollinger_windows=[20, 90, 180, 240],
                     lags=[1, 2, 4, 6, 12]):
    """
    Apply indicators, rolling volatilities, expanding skew, and Hilbert Transform indicators for each asset.
    
    :param data: DataFrame with weekly or monthly returns for each asset
    :param indicators: Dictionary of indicator functions
    :param assets: List of asset columns to process
    :param reference_data: DataFrame of daily returns for calculations
    :param theta: Smoothing parameter for EWMA
    :param initial_vol: Initial volatility for EWMA calculation
    :param rolling_windows: List of windows for calculating rolling volatilities
    :param lags: List of lag periods to create lagged variables
    """

    # Calculate indicators (MACD, RSI, Bollinger Bands) using daily data in reference_data
    macd_df = calculate_macd(reference_data, short_period=12, long_period=26, signal_period=9)
    rsi_df = calculate_rsi(reference_data, window=14)
    bollinger_df = calculate_bollinger_bands(reference_data, windows=bollinger_windows)

    # Calculate rolling statistics using daily data
    rolling_vol_dfs = {window: reference_data.rolling(window=window).std() for window in rolling_windows}
    rolling_skew_dfs = {window: reference_data.rolling(window=window).skew() for window in rolling_windows}
    rolling_kurt_dfs = {window: reference_data.rolling(window=window).kurt() for window in rolling_windows}

    # Expanding skew/kurtosis using daily data
    expanding_skew_df = reference_data.expanding(min_periods=(252 * 2)).skew().dropna()
    expanding_kurt_df = reference_data.expanding(min_periods=(252 * 2)).kurt().dropna()

    # Hilbert Transform using daily data
    hilbert_df = calc_hilbert(reference_data)

    # Loop through each asset and calculate indicators
    asset_dataframes = {}
    for asset in assets:
        indicator_dfs = []

        # Get MACD, RSI, and Bollinger Bands indicators for the asset
        macd_asset_df = macd_df[[f'MACD_Line_{asset}', f'Signal_Line_{asset}', f'MACD_Histogram_{asset}']]
        rsi_asset_df = rsi_df[[f'RSI_{asset}', f'Stoch_RSI_{asset}']]
        bollinger_asset_df = bollinger_df[[f'SMA{window}_{asset}' for window in bollinger_windows] + 
                                          [f'Upper{window}_{asset}' for window in bollinger_windows] +
                                          [f'Lower{window}_{asset}' for window in bollinger_windows]]

        # Append indicators
        indicator_dfs.extend([macd_asset_df, rsi_asset_df, bollinger_asset_df])

        # Apply custom indicators from the indicators dictionary
        for indicator_name, indicator_func in indicators.items():
            indicator_dfs.append(indicator_func(data[[asset]]))

        # Calculate EWMA volatility for each asset
        ewma_vol_df = calc_ewma_volatility(reference_data, asset, theta=theta, initial_vol=initial_vol)
        indicator_dfs.append(ewma_vol_df)

        # Add rolling and expanding stats
        for window, df in rolling_vol_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingVol_{window}_{asset}'}))
        for window, df in rolling_skew_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingSkew_{window}_{asset}'}))
        for window, df in rolling_kurt_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingKurt_{window}_{asset}'}))
        indicator_dfs.extend([expanding_skew_df[[asset]].rename(columns={asset: f'ExpandingSkew_{asset}'}),
                              expanding_kurt_df[[asset]].rename(columns={asset: f'ExpandingKurt_{asset}'})])

        # Add Hilbert Transform for each asset
        indicator_dfs.append(hilbert_df[[f'ht_dcperiod_{asset}', f'ht_dcphase_{asset}', f'inphase_{asset}', 
                                         f'sine_{asset}', f'ht_trendmode_{asset}']])

        # Combine indicators and apply lags
        combined_df = pd.concat(indicator_dfs, axis=1).dropna()
        lagged_features = [combined_df.shift(lag).add_suffix(f'_lag{lag}') for lag in lags]
        combined_df = pd.concat([combined_df] + lagged_features, axis=1).dropna()

        # Add one-hot encoding for the asset and binary target variable (optional)
        one_hot_df = pd.DataFrame(0, index=combined_df.index, columns=[f'Asset_{i+1}' for i in range(len(assets))])
        assets_list = list(assets)
        one_hot_df[f'Asset_{assets_list.index(asset) + 1}'] = 1
        combined_df = pd.concat([combined_df, one_hot_df], axis=1)

        combined_df.columns = [re.sub(f'_Asset \d+', '', col) if not col.startswith('Asset_') else col
                               for col in combined_df.columns]      

        asset_dataframes[asset] = combined_df
    
    return asset_dataframes

In [22]:
roc_periods = [1, 3, 6, 9, 12]
asset_columns = [col for col in equity_domestic_monthly_rets.columns if col.startswith('Asset')]

indicators = {
    'ROC': lambda df: calculate_roc(df, roc_periods)
}

asset_dataframes_master={}

for i_investment in range(equity_domestic_monthly_rets.shape[1]):
    data_in=pd.DataFrame(equity_domestic.iloc[:,i_investment]).dropna()
    equity_domestic_monthly_rets_in=pd.DataFrame(equity_domestic_monthly_rets.loc[data_in.index[0]:])
    columns_in=data_in.columns
    asset_dataframes_out = apply_indicators(
        data=equity_domestic_monthly_rets_in,             # Weekly returns data for each asset
        indicators=indicators,        # Dictionary of indicators
        assets=columns_in,         # List of asset columns
        reference_data=data_in   # Daily return data for indicators calculation
    )
    asset_dataframes_master[i_investment]=asset_dataframes_out

i_investment = 3
asset_name = 'Asset 4'

asset_dataframes = asset_dataframes_master[i_investment]
asset_dataframes['Asset 4']


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,RollingKurt_252_lag12,RollingKurt_504_lag12,ExpandingSkew_lag12,ExpandingKurt_lag12,ht_dcperiod_lag12,ht_dcphase_lag12,inphase_lag12,sine_lag12,ht_trendmode_lag12,Asset_1
Date,,,,,,,,,,,,,,,,,,,,,
2004-02-02,0.0017,0.0122,-0.0105,31.1062,0.0000,1.5303,1.4540,1.3760,1.3209,1.5925,...,3.2575,4.2021,0.7910,4.0282,21.6181,-5.0773,-0.0398,-0.0885,1.0000,1
2004-03-01,0.0059,0.0064,-0.0005,62.7761,0.7450,1.5268,1.4899,1.4048,1.3526,1.5813,...,3.3721,4.1785,0.7962,4.0813,15.3987,216.9152,-0.0076,-0.6006,0.0000,1
2004-04-01,0.0002,-0.0073,0.0075,56.7237,0.8440,1.4943,1.5025,1.4376,1.3869,1.5466,...,3.2520,4.1560,0.7957,4.0323,15.0386,275.7590,-0.0201,-0.9950,1.0000,1
2004-05-03,0.0163,0.0176,-0.0013,63.9316,0.3562,1.5722,1.5277,1.4689,1.4194,1.6077,...,2.9749,4.0900,0.7740,3.9452,15.1164,218.5704,0.0188,-0.6235,1.0000,1
2004-06-01,-0.0025,-0.0032,0.0007,63.0120,0.9308,1.5479,1.5336,1.4887,1.4428,1.5870,...,2.9205,3.9713,0.7591,3.8881,21.4201,131.1886,0.0285,0.7525,1.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.0179,0.0605,-0.0426,40.1298,0.4345,25.5796,24.6066,22.9975,22.9244,26.2228,...,0.3848,0.4687,0.0881,10.0381,15.0003,-19.2777,-0.1019,-0.3301,0.0000,1
2024-06-03,0.0662,0.2541,-0.1879,33.9398,0.0847,27.3472,25.5899,23.6305,23.3853,28.5122,...,0.1193,0.4226,0.0880,10.0123,26.3760,192.9315,-0.2741,-0.2238,1.0000,1
2024-07-01,0.2206,0.1840,0.0366,65.5482,0.7480,27.0946,26.2409,24.2763,23.7090,28.0880,...,0.0553,0.4459,0.0866,10.0263,22.0463,229.2860,-0.2701,-0.7580,1.0000,1


In [30]:
equity_domestic.iloc[:,3].dropna()


Date
2001-01-03   -0.0338
2001-01-04   -0.0471
2001-01-05    0.0231
2001-01-08    0.0138
2001-01-09   -0.0126
               ...  
2024-08-26   -0.0048
2024-08-27    0.0046
2024-08-28    0.0007
2024-08-29   -0.0004
2024-08-30    0.0092
Name: Asset 4, Length: 5952, dtype: float64

You can stop here as this is the ultimate output for the outer loop experiment ONLY for the equity domestic assets data.
***

In [23]:
data_path = os.path.dirname(os.getcwd()) + '/data'
os.makedirs(data_path, exist_ok=True)

# Export each asset DataFrame to an Excel file
for i, (asset, df) in enumerate(asset_dataframes.items(), start=1):
    filename = os.path.join(data_path, f'equity_domestic_{i}_features.xlsx')
    df.to_excel(filename, sheet_name=f'{asset}_features', index=True)
    print(f"Exported {asset} to {filename}")

Exported Asset 4 to /home/sam/projects/UniversityofChicago/data/equity_domestic_1_features.xlsx


***
## Indicators for Equity Global

In [24]:
roc_periods = [1, 3, 6, 9, 12]
asset_columns = [col for col in equity_global_monthly_rets.columns if col.startswith('Asset')]

indicators = {
    'ROC': lambda df: calculate_roc(df, roc_periods)
}

asset_dataframes = apply_indicators(
    data=equity_global_monthly_rets,             # Weekly returns data for each asset
    indicators=indicators,        # Dictionary of indicators
    assets=asset_columns,         # List of asset columns
    reference_data=equity_global   # Daily return data for indicators calculation
)

asset_dataframes[asset_columns[0]]

,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_43,Asset_44,Asset_45,Asset_46,Asset_47,Asset_48,Asset_49,Asset_50,Asset_51,Asset_52
Date,,,,,,,,,,,,,,,,,,,,,
2013-05-01,-0.0482,-0.0575,0.0093,42.0910,1.0000,2.3919,2.6223,2.8271,2.8449,2.5253,...,0,0,0,0,0,0,0,0,0,0
2013-06-03,-0.0077,-0.0006,-0.0071,44.9336,0.0651,2.4080,2.5237,2.7187,2.8069,2.5093,...,0,0,0,0,0,0,0,0,0,0
2013-07-01,-0.0409,-0.0480,0.0071,44.0236,1.0000,2.3021,2.4371,2.6008,2.7588,2.5227,...,0,0,0,0,0,0,0,0,0,0
2013-08-01,-0.0030,0.0049,-0.0079,48.3212,0.1021,2.2942,2.3591,2.5009,2.6877,2.4098,...,0,0,0,0,0,0,0,0,0,0
2013-09-03,-0.0077,-0.0102,0.0025,45.9418,0.4748,2.2674,2.3164,2.4773,2.5910,2.3805,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-03,-0.0571,-0.0428,-0.0143,13.6485,0.2340,1.0573,1.1544,1.1241,1.0860,1.2201,...,0,0,0,0,0,0,0,0,0,0
2023-02-01,0.0425,0.0401,0.0024,73.5761,0.3707,1.2537,1.0994,1.1363,1.1293,1.3712,...,0,0,0,0,0,0,0,0,0,0
2023-03-01,-0.0064,0.0054,-0.0119,35.2048,0.1974,1.2907,1.1704,1.1420,1.1483,1.3771,...,0,0,0,0,0,0,0,0,0,0


In [25]:
data_path = os.path.dirname(os.getcwd()) + '/data'
os.makedirs(data_path, exist_ok=True)

# Export each asset DataFrame to an Excel file
for i, (asset, df) in enumerate(asset_dataframes.items(), start=1):
    filename = os.path.join(data_path, f'equity_global_{i}_features.xlsx')
    df.to_excel(filename, sheet_name=f'{asset}_features', index=True)
    print(f"Exported {asset} to {filename}")

Exported Asset 1 to /home/sam/projects/UniversityofChicago/data/equity_global_1_features.xlsx
Exported Asset 2 to /home/sam/projects/UniversityofChicago/data/equity_global_2_features.xlsx
Exported Asset 3 to /home/sam/projects/UniversityofChicago/data/equity_global_3_features.xlsx
Exported Asset 4 to /home/sam/projects/UniversityofChicago/data/equity_global_4_features.xlsx
Exported Asset 5 to /home/sam/projects/UniversityofChicago/data/equity_global_5_features.xlsx
Exported Asset 6 to /home/sam/projects/UniversityofChicago/data/equity_global_6_features.xlsx
Exported Asset 7 to /home/sam/projects/UniversityofChicago/data/equity_global_7_features.xlsx
Exported Asset 8 to /home/sam/projects/UniversityofChicago/data/equity_global_8_features.xlsx
Exported Asset 9 to /home/sam/projects/UniversityofChicago/data/equity_global_9_features.xlsx
Exported Asset 10 to /home/sam/projects/UniversityofChicago/data/equity_global_10_features.xlsx
Exported Asset 11 to /home/sam/projects/UniversityofChicag

***

In [26]:
def apply_indicators_to_all_periods(monthly_returns, split_data, roc_periods):
    """
    Applies the apply_indicators function to each period's monthly returns.

    Args:
    monthly_returns (dict): Dictionary of monthly return dataframes for each period.
    split_data (dict): Dictionary of daily return data for each period.
    roc_periods (list): List of periods to use for Rate of Change (ROC) calculation.

    Returns:
    dict: A dictionary with the same keys as monthly_returns, where each value is the
          result of apply_indicators for that period.
    """
    all_period_indicators = {}
    
    indicators = {
        'ROC': lambda df: calculate_roc(df, roc_periods)
    }
    
    for period_name, monthly_data in monthly_returns.items():
        asset_columns = [col for col in monthly_data.columns if col.startswith('Asset')]
        
        if not asset_columns:
            print(f"Warning: No asset columns found for period {period_name}. Skipping.")
            continue
        
        try:
            asset_dataframes = apply_indicators(
                data=monthly_data,
                indicators=indicators,
                assets=asset_columns,
                reference_data=split_data.get(period_name)
            )

            all_period_indicators[period_name] = asset_dataframes

        except KeyError as e:
            print(f"KeyError: {e} in period {period_name}. Please verify column names and data format.")
            continue
    
    return all_period_indicators


roc_periods = [1, 3, 6, 9, 12]
monthly_dom_indicators = apply_indicators_to_all_periods(monthly_dom_returns, split_date_domestic, roc_periods)
monthly_glb_indicators = apply_indicators_to_all_periods(monthly_glb_returns, split_date_global, roc_periods)

print('Asset Split Guidance of Equity Domestic Basket based on the Available Assets for Certain Periods:')
for period_name, indicator_data in monthly_dom_indicators.items():
    print(f"Available assets in {period_name}: {list(indicator_data.keys())}")

print('\nAsset Split Guidance of Equity Global Basket based on the Available Assets for Certain Periods:')
for period_name, indicator_data in monthly_glb_indicators.items():
    print(f"Available assets in {period_name}: {list(indicator_data.keys())}")

NameError: name 'monthly_dom_returns' is not defined

In [ ]:
target_period = "equity_dom_period_4"
data_dicts = [monthly_dom_indicators, monthly_glb_indicators]

for i, data_dict in enumerate(data_dicts, start=1):
    if target_period in data_dict:
        print(f"\nData from dictionary {i} for period '{target_period}':")
        
        for asset, data in data_dict[target_period].items():
            print(f"\nAsset: {asset}")
            display(data)
    else:
        print(f"No data found in dictionary {i} for period '{target_period}'.")


Data from dictionary 1 for period 'equity_dom_period_4':

Asset: Asset 1


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_37,Asset_38,Asset_39,Asset_40,Asset_41,Asset_42,Asset_43,Asset_44,Asset_45,Asset_46
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-01,0.0023,0.0016,0.0006,68.4296,0.6953,0.0042,-0.0012,-0.0001,-0.0002,0.0272,...,0,0,0,0,0,0,0,0,0,0
2019-08-01,-0.0009,0.0006,-0.0016,51.4542,0.3782,0.0001,-0.0005,-0.0002,-0.0003,0.0262,...,0,0,0,0,0,0,0,0,0,0
2019-09-03,-0.0001,0.0001,-0.0002,43.8511,0.5122,-0.0005,-0.0014,-0.0002,-0.0006,0.0293,...,0,0,0,0,0,0,0,0,0,0
2019-10-01,-0.0025,-0.0012,-0.0013,27.9882,0.0000,0.0005,0.0001,-0.0002,-0.0003,0.0224,...,0,0,0,0,0,0,0,0,0,0
2019-11-01,-0.0001,0.0007,-0.0008,70.3539,0.6243,0.0019,0.0004,-0.0006,-0.0002,0.0222,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,-0.0021,-0.0013,-0.0008,29.1175,0.1816,-0.0031,0.0012,0.0005,0.0011,0.0179,...,0,0,0,0,0,0,0,0,0,0
2024-06-03,0.0004,0.0005,-0.0001,63.6930,0.3413,0.0017,0.0021,0.0011,0.0009,0.0186,...,0,0,0,0,0,0,0,0,0,0
2024-07-01,-0.0021,-0.0005,-0.0016,47.1518,0.0352,-0.0016,0.0011,0.0015,0.0004,0.0237,...,0,0,0,0,0,0,0,0,0,0



Asset: Asset 2


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_37,Asset_38,Asset_39,Asset_40,Asset_41,Asset_42,Asset_43,Asset_44,Asset_45,Asset_46
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-01,0.0024,0.0012,0.0012,48.6270,0.7419,0.0013,-0.0026,-0.0015,-0.0015,0.0266,...,0,0,0,0,0,0,0,0,0,0
2019-08-01,-0.0047,-0.0014,-0.0032,27.4410,0.0114,-0.0069,-0.0023,-0.0019,-0.0016,0.0323,...,0,0,0,0,0,0,0,0,0,0
2019-09-03,-0.0002,0.0000,-0.0002,30.9136,0.2695,-0.0040,-0.0039,-0.0020,-0.0023,0.0341,...,0,0,0,0,0,0,0,0,0,0
2019-10-01,-0.0025,0.0007,-0.0032,50.0850,0.0000,0.0078,-0.0012,-0.0017,-0.0015,0.0456,...,0,0,0,0,0,0,0,0,0,0
2019-11-01,-0.0035,-0.0019,-0.0016,51.5183,0.1217,0.0024,-0.0003,-0.0015,-0.0009,0.0446,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,-0.0012,-0.0004,-0.0008,35.2574,0.8234,-0.0040,-0.0002,-0.0001,0.0005,0.0217,...,0,0,0,0,0,0,0,0,0,0
2024-06-03,0.0009,0.0003,0.0007,48.7811,0.0953,0.0019,-0.0001,0.0001,0.0004,0.0239,...,0,0,0,0,0,0,0,0,0,0
2024-07-01,-0.0019,-0.0008,-0.0011,33.1621,0.0621,-0.0044,-0.0012,-0.0001,-0.0002,0.0196,...,0,0,0,0,0,0,0,0,0,0



Asset: Asset 3


...


Asset: Asset 45


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_37,Asset_38,Asset_39,Asset_40,Asset_41,Asset_42,Asset_43,Asset_44,Asset_45,Asset_46
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-01,-0.0005,-0.0003,-0.0002,63.2961,0.7211,0.0000,0.0011,0.0010,0.0005,0.0120,...,0,0,0,0,0,0,0,0,1,0
2019-08-01,0.0021,0.0004,0.0017,78.1933,1.0000,0.0005,0.0010,0.0011,0.0006,0.0157,...,0,0,0,0,0,0,0,0,1,0
2019-09-03,-0.0006,-0.0002,-0.0004,64.4261,0.0642,0.0031,0.0021,0.0014,0.0011,0.0234,...,0,0,0,0,0,0,0,0,1,0
2019-10-01,0.0007,0.0003,0.0005,60.3322,1.0000,-0.0012,0.0014,0.0011,0.0011,0.0207,...,0,0,0,0,0,0,0,0,1,0
2019-11-01,0.0011,0.0002,0.0009,47.2685,0.8210,-0.0018,0.0008,0.0009,0.0010,0.0143,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.0008,0.0001,0.0007,44.5365,1.0000,-0.0017,-0.0011,-0.0002,-0.0004,0.0165,...,0,0,0,0,0,0,0,0,1,0
2024-06-03,0.0012,-0.0000,0.0013,58.4266,0.4506,0.0012,0.0000,0.0001,-0.0003,0.0172,...,0,0,0,0,0,0,0,0,1,0
2024-07-01,-0.0031,-0.0010,-0.0021,47.3030,0.0000,0.0001,-0.0001,0.0003,-0.0003,0.0225,...,0,0,0,0,0,0,0,0,1,0



Asset: Asset 46


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_37,Asset_38,Asset_39,Asset_40,Asset_41,Asset_42,Asset_43,Asset_44,Asset_45,Asset_46
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-01,-0.0001,-0.0000,-0.0001,66.8085,0.3562,0.0001,0.0002,0.0002,0.0002,0.0018,...,0,0,0,0,0,0,0,0,0,1
2019-08-01,0.0002,0.0000,0.0002,71.0269,1.0000,0.0001,0.0002,0.0002,0.0002,0.0021,...,0,0,0,0,0,0,0,0,0,1
2019-09-03,0.0000,-0.0000,0.0000,67.9964,0.4304,0.0002,0.0002,0.0002,0.0002,0.0020,...,0,0,0,0,0,0,0,0,0,1
2019-10-01,0.0002,0.0001,0.0001,64.2045,1.0000,-0.0000,0.0002,0.0002,0.0002,0.0020,...,0,0,0,0,0,0,0,0,0,1
2019-11-01,0.0001,0.0000,0.0001,61.5008,0.8441,-0.0001,0.0001,0.0002,0.0002,0.0013,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.0001,0.0000,0.0001,59.8433,1.0000,-0.0001,0.0000,0.0002,0.0001,0.0025,...,0,0,0,0,0,0,0,0,0,1
2024-06-03,0.0001,0.0000,0.0001,64.2311,0.4841,0.0002,0.0001,0.0002,0.0001,0.0018,...,0,0,0,0,0,0,0,0,0,1
2024-07-01,-0.0001,-0.0000,-0.0001,68.7543,0.6241,0.0002,0.0001,0.0002,0.0002,0.0023,...,0,0,0,0,0,0,0,0,0,1


No data found in dictionary 2 for period 'equity_dom_period_4'.


In [ ]:
target_period = "equity_glb_period_1"
data_dicts = [monthly_dom_indicators, monthly_glb_indicators]

for i, data_dict in enumerate(data_dicts, start=1):
    if target_period in data_dict:
        print(f"\nData from dictionary {i} for period '{target_period}':")
        
        for asset, data in data_dict[target_period].items():
            print(f"\nAsset: {asset}")
            display(data)
    else:
        print(f"No data found in dictionary {i} for period '{target_period}'.")

No data found in dictionary 1 for period 'equity_glb_period_1'.

Data from dictionary 2 for period 'equity_glb_period_1':

Asset: Asset 1


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_9,Asset_10,Asset_11,Asset_12,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18
Date,,,,,,,,,,,,,,,,,,,,,



Asset: Asset 2


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_9,Asset_10,Asset_11,Asset_12,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18
Date,,,,,,,,,,,,,,,,,,,,,



Asset: Asset 4


...

,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_9,Asset_10,Asset_11,Asset_12,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18
Date,,,,,,,,,,,,,,,,,,,,,



Asset: Asset 51


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_9,Asset_10,Asset_11,Asset_12,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18
Date,,,,,,,,,,,,,,,,,,,,,



Asset: Asset 52


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_9,Asset_10,Asset_11,Asset_12,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18
Date,,,,,,,,,,,,,,,,,,,,,
